In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [3]:
df = pd.read_excel("OppScrData.xlsx", sheet_name="1st CT Data")

In [4]:
data = df.iloc[:].to_numpy()
N = len(data)   # num of samples
# each column contains an outcome, zeroth column being death
outcomes = data[:, 15:40]
# get death outcomes as labels TODO add other outcomes
y = np.array([0 if np.isnan(outcomes[:, 0][i]) else 1 for i in range(N)]) 
X_ct = ct_data = data[:, 41:52]     # get CT data as features
X_cln = cln_data = data[:, 3:14]    # get clinical data as features

In [5]:
# Preprocess CT data.
# last column of CT data contains ' ', convert them to NaN
# then CT data only have types nan, float, and int
for i in range(N):
    if X_ct[i,-1] == ' ':
        X_ct[i,-1] = np.nan
# eliminate NaN data by imputing them using mean strategy
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_ct)
X_ct = imp.transform(X_ct)
# standardize
scaler = StandardScaler()
scaler.fit(X_ct) 
X_ct = scaler.transform(X_ct)

In [6]:
# Preprocess clinical data.
# convert string to numbers/nan
for i in range(N):
    X_cln[i,2] = 1 if X_cln[i,2] == 'Y' else 0
    X_cln[i,3] = 1 if X_cln[i,3] == 'Male' else 0
    X_cln[i,5] = 1 if X_cln[i,5] == 'Yes' else 0
    # X[i,6] TODO alchohol?
    """
    Risk is considered low if the FRS is less than 10%, moderate if it is
    10% to 19%, and high if it is 20% or higher. 

    Ref: 
    Bosomworth NJ. Practical use of the Framingham risk score in 
    primary prevention: Canadian perspective. Can Fam Physician. 2011 Apr;
    57(4):417–23. http://www.cfp.ca/cgi/pmidlookup?view=long&pmid=21626897
    """
    # we could classify FRS into 3 categories: low, moderate, and high
    # but here we convert them to float, taking precise values
    if X_cln[i,7] == 'X':
        X_cln[i,7] = np.nan
    elif type(X_cln[i,7]) is str:  
        if X_cln[i,7][0] == '<' or X_cln[i,7][0] == '>':
            # take the middle value of [0,x%]
            X_cln[i,7] = float(X_cln[i,7][1:-1])/200 

    if X_cln[i,8] == '_':
        X_cln[i,8] = np.nan 

# eliminate Clinical F/U interval since its irrelevant is ambiguous
# eliminate BMI > 30 and FRAX 10y Hip Fx Prob. 
#   since they overlap with BMI and FRAX 10y
# eliminate Alcohol abuse since it lacks too many samples
# eliminate Met sx since it's really more of an outcome, and it lacks 
#   data points TODO not eliminate them?
clms = np.concatenate((np.array([1]), np.arange(3,6), \
    np.arange(7,9)))
X_cln = X_cln[:, clms] 
# deal with NaN by imputing them as mean values.
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp.fit(X_cln)
X_cln = imp.transform(X_cln)
# standardize
scaler = StandardScaler()
scaler.fit(X_cln) 
X_cln = scaler.transform(X_cln)

In [7]:
# Combine both CT and clinical data.
X = np.hstack((X_ct, X_cln))

In [8]:
# Cross validation
# scores of each loop
sc_ct = np.array([])
sc_cln = np.array([])
sc_all = np.array([])
n = 3 # Reduce n to shorten time.
train_size = 0.6    # ((num of training samples) / N)
clf = RandomForestClassifier()  # for temporary use 
for i in range(n):
    X_ct_train, X_ct_test, y_train, y_test = \
        train_test_split(X_ct, y, test_size=1-train_size)
    clf.fit(X_ct_train, y_train)
    sc_ct = np.append(sc_ct, clf.score(X_ct_test, y_test))

    X_cln_train, X_cln_test, y_train, y_test = \
        train_test_split(X_cln, y, test_size=1-train_size)
    clf.fit(X_cln_train, y_train)
    sc_cln = np.append(sc_cln, clf.score(X_cln_test, y_test))

    X_train, X_test, y_train, y_test = \
        train_test_split(X, y, test_size=1-train_size)
    clf.fit(X_train, y_train)
    sc_all = np.append(sc_all, clf.score(X_test, y_test))
    
print("Cross validation accuracy:")
print("CT only       : ", np.mean(sc_ct))
print("Clinical only : ", np.mean(sc_cln))
print("Both          : ", np.mean(sc_all))

Cross validation accuracy:
CT only       :  0.9408310749774165
Clinical only :  0.933694670280036
Both          :  0.939476061427281


TODO ?

In [9]:
# Random forest classifier for CT data
rfclf = RandomForestClassifier()
rfclf.fit(X_ct, y)

RandomForestClassifier()

In [11]:
# R2 score:
X_train, X_test, y_train, y_test = \
    train_test_split(X_ct, y, test_size=1-train_size)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_ct)
print("r2 score: ", r2_score(y, y_pred))

r2 score:  0.5564772220899257


In [ ]:
# TODO? Death outcome includes [days from CT]?
# TODO? Integrate data of same category (e.g. Cols AQ-AU: Fat measures))?
# TODO? Clinical training data includes [F/U d from CT]?
# TODO? Predict other outcomes as well (including the Metabolic sx?)?